# Model tunning
**Finding best hyperparameters and properly setups for Model**

I

In [ ]:
# Setup

import numpy as np
import pandas as pd

from pkg import utils
from pkg import tuning

In [ ]:
# Importing data

parent = 'data'
filename = 'featured.csv'

path = utils.get_path(parent, filename)

df = pd.read_csv(path, index_col=0)